<a href="https://colab.research.google.com/github/xinhe-zhang/heatwave/blob/main/HW2_%E5%BC%B5%E6%98%95%E8%8D%B7_R12228016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **IPCS Heatwave HW1**
張昕荷 地理碩一 R12228016

***

Search two heatwave news, plot it and write what you have observed.
## **Event 1 : 2020 Siberia Heatwave**

In the first six months of 2020, Siberia experienced a period of unusually high temperatures, including a record-breaking 38 degrees C in the town of Verkhoyansk on 20 June, causing wide-scale impacts including wildfires, loss of permafrost, and an invasion of pests.

Reference:

https://www.worldweatherattribution.org/\



In [1]:
%matplotlib inline
import xarray as xr
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn
import sklearn.ensemble
import scipy.stats
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

!pip install cartopy
import cartopy.crs as ccrs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 87.6 MB/s eta 0:00:00


In [2]:
#Download ERA5 Tmax, 1979-2022, land only, 144*73
#I've changed the data to my own drive.
!gdown https://drive.google.com/uc?id=14okWeTrUH1nHZcgbmBZupRM6s5EcVoYu&export=download

Downloading...
From: https://drive.google.com/uc?id=14okWeTrUH1nHZcgbmBZupRM6s5EcVoYu
To: /content/ERA5_SFC_mx2t_1979-2022_r144x73_day.land.nc
100% 676M/676M [00:03<00:00, 169MB/s]


In [4]:
filename ='ERA5_SFC_mx2t_1979-2022_r144x73_day.land.nc'
ds_mx2t = xr.open_dataset(filename)

In [5]:
# Convert Kelvin to Celsius
ds_mx2t = ds_mx2t - 273.15

In [6]:
ds_mx2t['time'] = ds_mx2t['time'].astype('datetime64[ns]')
print(ds_mx2t['time'].dtype)
print(ds_mx2t['time'])

start_date = np.datetime64('1979-01-01')
end_date = np.datetime64('2023-01-01')  # Adjust the end date as needed

# Create a time array with intervals of 1 day
time_array = np.arange(start_date, end_date, np.timedelta64(1, 'D'))

print(time_array)
ds_mx2t['time'] = time_array

datetime64[ns]
<xarray.DataArray 'time' (time: 16071)>
array(['1970-01-01T00:00:00.019790101', '1970-01-01T00:00:00.019790102',
       '1970-01-01T00:00:00.019790103', ..., '1970-01-01T00:00:00.020221229',
       '1970-01-01T00:00:00.020221230', '1970-01-01T00:00:00.020221231'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T00:00:00.019790101 ... 1970-01-...
Attributes:
    standard_name:  time
    long_name:      Date
    units:          day as %Y%m%d.%f
    calendar:       proleptic_gregorian
    axis:           T
['1979-01-01' '1979-01-02' '1979-01-03' ... '2022-12-29' '2022-12-30'
 '2022-12-31']


In [7]:
# check the data size
mx2t=ds_mx2t['mx2t']
print(mx2t.shape)

(16071, 73, 144)
